In [1]:
import os
import torch
# import wandb

import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import GroupKFold
folds = GroupKFold(n_splits=5)

In [3]:
IMG_SIZE = 512
TRAIN_PATH = f'./tmp_{IMG_SIZE}/train/'
BATCH_SIZE = 16
EPOCHS = 10

In [4]:
df = pd.read_csv('./siim-covid19-detection/train_image_level.csv')
df.head()


,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [4]:
df.id = df.apply(lambda row: row.id.split('_')[0], axis=1)
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.png', axis=1)
df['image_level']  = df.apply(lambda row: row.label.split(' ')[0],axis=1)
df.head()

,id,boxes,label,StudyInstanceUID,path,image_level
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,./tmp_640/train/000a312787f2.png,opacity
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,./tmp_640/train/000c3a3f293f.png,none
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,./tmp_640/train/0012ff7358bc.png,opacity
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,./tmp_640/train/001398f4ff4f.png,opacity
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,./tmp_640/train/001bd15d1891.png,opacity


In [5]:
meta_df = pd.read_csv('./meta.csv')
# train_meta_df = meta_df.loc[meta_df.split=="train"]


In [6]:
train_meta_df = meta_df[meta_df.split=='train']

In [7]:
train_meta_df = train_meta_df.drop('split',axis=1)

In [8]:
train_meta_df.columns = ['id','dim0','dim1']

In [9]:
train_meta_df

,id,dim0,dim1
1263,1b3d36c26846,2330,2718
1264,e6215d0188e5,2999,2998
1265,691e08460a87,2991,2992
1266,92433ed035bd,3480,4240
1267,fe94f73e3072,4020,4891
...,...,...,...
7592,12c435a26db5,2800,3408
7593,a06bc359b5fb,2800,3408
7594,4ed22347ea95,3480,4248
7595,f071f42fe94e,3198,3056


In [10]:
df = df.merge(train_meta_df, on='id',how='left')

In [11]:
df

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,./tmp_640/train/000a312787f2.png,opacity,3488,4256
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,./tmp_640/train/000c3a3f293f.png,none,2320,2832
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,./tmp_640/train/0012ff7358bc.png,opacity,2544,3056
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,./tmp_640/train/001398f4ff4f.png,opacity,3520,4280
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,./tmp_640/train/001bd15d1891.png,opacity,2800,3408
...,...,...,...,...,...,...,...,...
6329,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,./tmp_640/train/ffcc6edd9445.png,none,4240,3480
6330,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,./tmp_640/train/ffd91a2c4ca0.png,none,2800,3408
6331,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,./tmp_640/train/ffd9b6cf2961.png,opacity,2388,3050
6332,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,./tmp_640/train/ffdc682f7680.png,opacity,3488,4256


In [12]:
from sklearn.model_selection import train_test_split


train_df,valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df.image_level.values)

In [13]:
train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

In [14]:
df = pd.concat([train_df, valid_df]).reset_index(drop=True)

In [15]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')


Size of dataset: 6334, training images: 5067. validation images: 1267


In [16]:
os.makedirs(f'data_{IMG_SIZE}/train/images', exist_ok=True)
os.makedirs(f'data_{IMG_SIZE}/valid/images', exist_ok=True)


In [17]:
os.makedirs(f'data_{IMG_SIZE}/train/labels', exist_ok=True)
os.makedirs(f'data_{IMG_SIZE}/valid/labels', exist_ok=True)


In [18]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'data_{IMG_SIZE}/train/images/{row.id}.png')
    else:
        copyfile(row.path, f'data_{IMG_SIZE}/valid/images/{row.id}.png')

  0%|          | 0/6334 [00:00<?, ?it/s]

In [19]:
os.listdir(f'./tmp_{IMG_SIZE}/train')[:5]

['7183445f4d28.png',
 '1ea96fd9dd7c.png',
 '273722673feb.png',
 '3566e20a178e.png',
 '8d7cfdfc8bf2.png']

In [20]:
row.path

'./tmp_640/train/d7a2083fec46.png'

In [21]:
import yaml

data_yaml = dict(
    train = f'/home/outerform/github-repo/DeepLearningCourse-Project/data_{IMG_SIZE}/train/images',
    val = f'/home/outerform/github-repo/DeepLearningCourse-Project/data_{IMG_SIZE}/valid/images',
    nc = 2,
    names = ['none', 'opacity']
)

with open(f'data_{IMG_SIZE}.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    


In [22]:
def get_bbox(row):
    bboxes = []
    bbox = []
    labels  = row.label.split(' ')
    bboxes = []
    for i in range(0,len(labels), 6):
        # print(labels[i+2:i+6])
        bboxes.append(list(map(float,labels[i+2:i+6])))
    return np.array(bboxes)

        
        

In [23]:
def scale(row,bboxes):
    scalex = IMG_SIZE/row.dim1
    scaley = IMG_SIZE/row.dim0
    scaled_bboxes = np.zeros_like(bboxes)
    scaled_bboxes[:,[0,2]] = bboxes[:,[0,2]]*scalex
    scaled_bboxes[:,[1,3]] = bboxes[:,[1,3]]*scaley
    return scaled_bboxes

In [24]:
def yolo_format(bboxes):
    x = ((bboxes[:,0]+bboxes[:,2])/2)/IMG_SIZE
    y = ((bboxes[:,1]+bboxes[:,3])/2)/IMG_SIZE
    width = (bboxes[:,2]-bboxes[:,0])/IMG_SIZE
    height = (bboxes[:,3]-bboxes[:,1])/IMG_SIZE
    return np.column_stack([x,y,width,height]).astype(np.float32).reshape(-1,4)

In [25]:
def convert_to_yolo(row):
    bboxes = get_bbox(row)
    scaled_bboxes = scale(row,bboxes)
    yolo_bboxes = yolo_format(scaled_bboxes)
    # image_id = row.id
    # label = [0]*len(yolo_bboxes)
    return yolo_bboxes

In [26]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # print(row.split)
    label = row.image_level
    if row.split == 'train':
        label_path = f'data_{IMG_SIZE}/train/labels/{row.id}.txt'
    else:
        label_path = f'data_{IMG_SIZE}/valid/labels/{row.id}.txt'
    
    if label == 'opacity':
        yoloboxes = convert_to_yolo(row)
        with open(label_path, 'w') as f:
            for bbox in yoloboxes:
                f.write(f'1 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n')


  0%|          | 0/6334 [00:00<?, ?it/s]

In [27]:
x = np.array([1,2])
y= np.array([1,3])